# Notebook #1: Extracción

- En este notebook, extraeremos datos haciendo uso de APIs, para las sigueintes fuentes de información:
1. API Geoapify: datos geográficos de una ciudad.
2. API Idealista: datos de viviendas en venta y alquiler.

- Todas las funciones aquí utilizadas encuentran su soporte en [este soporte](../src/soporte_ETL.py).

- Dado que los datos se modifican cada vez que se ejecuta una función de consulta, las mismas están comentadas. Para ejecutarlas, debe eliminarse la #, con la consecuencia de que, los datos de origen serán sustituidos.

- El primer paso será importar las librerías necesarias:

In [1]:
%load_ext autoreload
%autoreload 2

# Librerías para tratamiento de datos

import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None) # Parámetro que modifica la visualización de los DFs

# Librerías para el acceso a variables y funciones
import sys
sys.path.append("../")
import src.soporte_extraccion as setl
import src.soporte_mongo as sm

# Librerías para trabajar con distintos formatos de archivos
import pickle
import json

# Librería para ignorar avisos
import warnings
warnings.filterwarnings("ignore") # Ignora TODOS los avisos

- Por medio de la API de Geoapify, se extrae la información geográfica de las divisiones administrativas de Zaragoza, en este caso, los distritos.
- La función `geoconsulta_distritos` recibe como único argumento el ID de la división administrativa deseada, en este caso, la ciudad de Zaragoza.
- El ID se obtiene a través del endpoint Geocoding, dentro de la misma API: https://apidocs.geoapify.com/playground/geocoding/.
- Una vez extraída, se guarda en un archivo json.

In [13]:
response_distritos = setl.geoconsulta_distritos("516ca3dd0b861fedbf594631421e8bd84440f00101f9018c46050000000000c002069203085a617261676f7a61")
#with open("../data/raw/distritos_zaragoza.json", "w") as f:
#    json.dump(response_distritos, f, indent=4)

- Carga del json genrado:

In [14]:
with open("../data/raw/distritos_zaragoza.json", "r") as f:
    response_distritos = json.load(f)

## Creación del DF Distritos

- Usando la función `dataframe_distritos`, transformaremos el JSON obtenido en la respuesta en un GeoDataFrame con los datos geográficos de cada uno de los 16 distritos, que convertiremos a geografía de multipolígono.
- Como paso final, guardamos el archivo.

In [15]:
gdf_distritos = setl.dataframe_distritos(response_distritos)
#gdf_distritos.to_file("../data/transformed/gdf_distritos.geojson", driver="GeoJSON")

- Comprobamos la apariencia del nuevo GDF y los tipos de datos.

In [16]:
gdf_distritos.head(5)

,distrito,geometry
0,Distrito Rural,"MULTIPOLYGON (((-0.9212 41.50517, -0.91835 41...."
1,Sur,"MULTIPOLYGON (((-1.06057 41.61549, -1.05914 41..."
2,Miralbueno,"MULTIPOLYGON (((-0.98545 41.65679, -0.98502 41..."
3,Santa Isabel,"MULTIPOLYGON (((-0.8438 41.67621, -0.84371 41...."
4,El Rabal,"MULTIPOLYGON (((-0.88153 41.65838, -0.87771 41..."


In [17]:
gdf_distritos.dtypes

distrito      object
geometry    geometry
dtype: object

### Subida a Mongo

In [18]:
# bd = sm.conectar_a_mongo("ProyectoRentabilidad")

# sm.subir_geodataframe_a_mongo(bd, gdf_distritos, "distritos")

# bd.client.close()

GeoDataFrame subido a la colección: distritos


## API Idealista

- Obtendremos datos de viviendas en venta y en alquiler a través de una API de idealista.
- De la consulta, que realizamos con la función `setl.consulta_idealista()`, obtendremos una lista correspondiente a cada página de respuesta.
- La función recibe como argumentos el tipo de operación, el código y el nombre de la ciudad que deseamos consultar, precio mínimo, máximo y el número de páginas deseado, devolviendo 40 resultados por página. La lista de resultados la almacenaremos en un archivo de tipo `json`.
- El locationID se puede obtener haciendo una consulta al endpoint https://rapidapi.com/scraperium/api/idealista7/playground/apiendpoint_1c6db49a-0793-4aa7-840b-6b8fc8868c3a.
- Dadas las limitaciones de la API, cada iteración tarda unos 10 segundos.
- Realizaremos dos consultas, una para las operaciones de venta y otra para las de alquiler.
- El número de páginas se ha elegido realizando una consulta manual del número de anuncios publicados en Enero 2025 en la plataforma, sabiendo que, cada página contiene un máximo de 40 resultados. En este caso, hay 452 anuncios de alquiler entre 400€ y 1.500€ euros, y 586 de venta entre 60.000€ y 150.000€.

In [2]:
resultados_idealista_sale = setl.consulta_idealista("sale", "0-EU-ES-50-17-001-297", "Zaragoza", "60000", "150000", 12)

100%|██████████| 12/12 [01:21<00:00,  6.77s/it]


In [8]:
#resultados_idealista_rent = setl.consulta_idealista("rent", "0-EU-ES-50-17-001-297", "Zaragoza", "400", "1500", 15)

100%|██████████| 15/15 [01:21<00:00,  5.42s/it]


In [4]:
# with open("../data/raw/idealista-sale2.json", "w") as json_file:
#     json.dump(resultados_idealista_sale, json_file, indent=4)

# # with open("../data/raw/idealista-rent.json", "w") as json_file:
# #     json.dump(resultados_idealista_rent, json_file, indent=4)

- Ahora importaremos el resultado de la consulta, comprobaremos que la longitud de la lista sea correcta y le aplicaremos la función `dataframe_idealista()`, que recibe como argumento el archivo json con los resultados y devuelve un dataframe con los datos transformados.

In [4]:
with open("../data/raw/idealista-sale.json", 'r') as file:
    resultados_idealista_sale = json.load(file)

with open("../data/raw/idealista-rent.json", 'r') as file:
    resultados_idealista_rent = json.load(file)

In [68]:
print(f""" Páginas resultados venta: {len(resultados_idealista_sale)}
Páginas resultados alquiler: {len(resultados_idealista_rent)} """)

 Páginas resultados venta: 12
Páginas resultados alquiler: 15 


Cada dataframe se compone de 28 columnas, descritas en el documento de Markdown disponible en [este enlace](../data/descripcion_variables.md).

In [5]:
df_idealista_sale = setl.dataframe_idealista(resultados_idealista_sale)
df_idealista_sale.head(1)

,codigo,latitud,longitud,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,parking,estado,direccion,descripcion,fecha,anunciante,contacto,cantidad_imagenes,urls_imagenes,tags_imagenes
0,104792745,41.697302,-0.869352,149900.0,1180.0,flat,True,4,True,127.0,3,2,False,False,False,True,True,good,"carretera de Huesca, 21","Junto a la Academia General Militar, en carret...",None,"Fincas Ruiz, Jose",876 21 08 84,39,[https://img4.idealista.com/blur/WEB_LISTING-M...,"[views, livingRoom, kitchen, facade, livingRoo..."


In [7]:
df_idealista_rent = setl.dataframe_idealista(resultados_idealista_rent)
df_idealista_rent.head(1)

,codigo,latitud,longitud,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,parking,estado,direccion,descripcion,fecha,anunciante,contacto,cantidad_imagenes,urls_imagenes,tags_imagenes
0,107028216,41.681202,-1.024342,750.0,6.0,flat,True,2,True,123.0,2,1,False,False,True,False,True,good,avenida Jota,En Garrapinillos tienes un Esplendido piso sem...,2025-01-15,"Re/Max Y Tú, Susana",876 21 09 00,39,[https://img4.idealista.com/blur/WEB_LISTING-M...,"[livingRoom, corridor, room, room, room, room,..."


- Vamos a crear un diccionario para traducir los nombres de los tipos de vivienda a español, y usamos el método replace para hacer los cambios en ambos dataframes.

In [6]:
diccionario_remplazo_tipo = {
    "flat": "piso",
    "duplex": "dúplex",
    "penthouse": "ático",
    "studio": "estudio",
    "countryHouse": "casa de campo",
    "chalet": "chalet"
}

In [7]:
df_idealista_sale["tipo"] = df_idealista_sale["tipo"].replace(diccionario_remplazo_tipo)
df_idealista_rent["tipo"] = df_idealista_rent["tipo"].replace(diccionario_remplazo_tipo)

- Comprobamos el total de resultados de la consulta:

In [10]:
print(f""" Venta: {df_idealista_sale.shape[0]}
Alquiler: {df_idealista_rent.shape[0]} """)

 Venta: 480
Alquiler: 450 


- Guardamos los archivos resultantes en formato CSV.
- Con esta tarea, finalizamos la extracción y transformación inicial de los datos. Continuaremos en el notebook #2 con el EDA, transformación y preprocesamiento.

In [8]:
df_idealista_sale.to_csv("../data/raw/idealista_sale2.csv")
df_idealista_rent.to_csv("../data/raw/idealista_rent.csv")